In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, LSTM, MaxPool2D, Flatten, InputLayer, Reshape, TimeDistributed
from tensorflow.keras.models import Sequential

def create_model():
    model = Sequential([
        InputLayer(input_shape=(600,160,6)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (1,5), strides=(1,4), activation='relu'),
        Conv2D(1, (1,4), activation='relu'),
        Flatten(),
        Dense(600, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 600, 160, 64)      3520      
                                                                 
 conv2d_41 (Conv2D)          (None, 600, 160, 64)      36928     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 300, 80, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 300, 80, 64)       36928     
                                                                 
 conv2d_43 (Conv2D)          (None, 300, 80, 64)       36928     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 150, 40, 64)      0         
 g2D)                                                 